In [1]:
import pathlib
import os
import random
import time
import timeit
import functools
from collections import OrderedDict
import requests
import sys

from time import sleep

### Function basics

In [107]:
def func(x, y, z):
    print(x, y, z)

func(1, 2, 3)
func(x=1, y=2, z=3)

1 2 3
1 2 3


In [111]:
def func(x='abc', y=42, z=None):
    print(x, y, z)


func()
func('xyz')
# func(, 43)
func(y=43)
func('xyz', 43, (0,1))

abc 42 None
xyz 42 None
abc 43 None
xyz 43 (0, 1)


In [2]:
class BaseModelForm(BaseForm):
    def __init__(self, data=None, files=None, auto_id='id_%s', prefix=None,
                 initial=None, error_class=ErrorList, label_suffix=None,
                 empty_permitted=False, instance=None, use_required_attribute=None,
                 renderer=None):
        ...
        
BaseModelForm(renderer='test')

NameError: name 'BaseForm' is not defined

In [118]:
def func(*args):
    print(args)
    return args

func('xyz')
func('xyz', 43, (0,1))
func(1,2,3)

('xyz',)
('xyz', 43, (0, 1))
(1, 2, 3)


(1, 2, 3)

In [119]:
print(type("x"))
print(type(func('xyz'))) 
print(type(func))

<class 'str'>
('xyz',)
<class 'tuple'>
<class 'function'>


In [123]:
def func(*args, **kwargs):
    print('args:', args, ', kwargs:', kwargs)


func()
func('xyz')
func('xyz', 43, (0,1))
func(y=43, x=45, parameter=45)

args: () , kwargs: {}
args: ('xyz',) , kwargs: {}
args: ('xyz', 43, (0, 1)) , kwargs: {}
args: () , kwargs: {'y': 43, 'x': 45, 'test': 12}


In [125]:
# keyword args
# could be default
def func(x, y, *, z):
    print(x, y, z)

# func(1, 2, 3)    
func(1, 2, z=3)
    

1 2 3


In [126]:
def func(x, y, *args, z):
    print(x, y, args, z)

func(1, 2, 3, 4, z=12)    
# func(1,2,z=3)
    

1 2 (3, 4) 12


In [127]:
def append_numbers(a, b):
    local_list = []
    local_list.append(a)
    local_list.append(b)
    print(local_list)

In [130]:
def append_numbers(a, b, *, validator=None):
    local_list = []
    local_list.append(a)
    local_list.append(b)
    if validator:
        print("I am using validator")
    print(local_list) 
     
append_numbers(1, 2, validator=3)


TypeError: append_numbers() takes 2 positional arguments but 3 were given

In [132]:
def func(*,x, y, z):
    print(x, y, z)

# def calculate_salary(a, b, c )
#     return a * b / c 
func(a=1,b=4,c=7)

TypeError: func() got an unexpected keyword argument 'a'

In [136]:
def func(x, y, z, /):
    
    print(x, y, z)
    
func(1,2,3)

1 2 3


In [141]:
# backward compatible
# def check_variable(*x, /):
    
def check_variable(x, /):
    if not x:
        raise ValueError(f"should be variable with value")

check_variable(0)

ValueError: should be variable with value

In [143]:
def _mod(x, y, /, *, mod):
    return (x + y) % mod

_mod(3, 5, mod=3)

0

In [46]:
# pos is faster 


In [ ]:
# def func(x:str) -> None:
#     pass

# x + func()

### Everything is an object, so is a function

In [144]:
def func(x='abc', y=42, z=None):
    print(x, y, z)

func1 = func
type(func1)

function

In [145]:
print(id(func1), id(func))
func1()

1709652464624 1709652464624
abc 42 None


<img src=https://miro.medium.com/max/700/1*n0MVfelkQrJ_zBym4b3CSA.png>

<img src=https://i.imgur.com/aye8Juy.png width=700>


In [146]:
def func(f):
    f()

def foo():
    print('Hello from foo')

def bar():
    print('Hello from bar')

func(foo)
func(bar)

Hello from foo
Hello from bar


In [148]:
def foo():
    print('Hello from foo')

# transparent 
def func(f):
#     print("Returning func f")
    return f

result = func(foo)
print(type(result))
foo()
result()


Returning func f
<class 'function'>
Hello from foo
Hello from foo


In [149]:
def hello_factory():
    def internal():
        print('Hello from internal')
    return internal

func = hello_factory()
func()
# internal()



Hello from internal


In [150]:
def transparent_factory(f):
    def internal():
        f()       
    return internal

foo_transparent = transparent_factory(foo)
foo_transparent()

# foo()

Hello from foo


In [154]:
def hello_factory_2(f):
    def internal():
        print(f">>> Is about to call '{f.__name__}'\n")
        f()
        print(f"\n>>> Just called '{f.__name__}'")
        
    return internal

foo_verbosed = hello_factory_2(foo)
foo_verbosed()

foo()

>>> Is about to call 'foo'

Hello from foo

>>> Just called 'foo'
Hello from foo


In [155]:
def foo():
    time.sleep(2)
    print('Hello from foo')

def profile(f):
    def internal():
        start = time.time()
        f()
        finish = time.time()
        print(f'Elapsed time: {finish - start}s')
    return internal

foo_profiled = profile(foo)
foo_profiled()

foo()


Hello from foo
Elapsed time: 2.002000331878662s
Hello from foo


### Decorators

#### D. is a function that adds some additional functionality, typically not changing original behaviour.

<img src=https://d33wubrfki0l68.cloudfront.net/12c8a296cc396d418b5407a4a4c6f9fd7d85f597/e8a54/wp-content/uploads/2018/06/python-decorator.png>

In [156]:
def profile(f):
    def internal():
        start = time.time()
        f()
        print(f'Elapsed time ({f.__name__}): {time.time() - start}s')
    return internal

foo_profiled = profile(foo)
foo_profiled()


Hello from foo
Elapsed time (foo): 2.006997585296631s


In [157]:
def fetch_url():
    res = requests.get('https://google.com')
    print('Content: \t', res.content[:100])

In [158]:
fetch_url_profiled = profile(fetch_url)
fetch_url_profiled()

Content: 	 b'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="uk"><head><meta content'
Elapsed time (fetch_url): 0.32297301292419434s


In [ ]:
1. add try except 

In [ ]:
2. add input parameteres  

In [ ]:
3. add return

In [159]:
def fetch_url(url):
    res = requests.get(url)
    print(f'\nContent for "{url}":\t ', res.content[:100])

def profile(f):
    def internal(url):
        start = time.time()
        f(url)
        print(f'Elapsed time ({f.__name__}): {time.time() - start}s')
    return internal

fetch_url_profiled = profile(fetch_url)
fetch_url_profiled('https://google.com')
fetch_url_profiled('https://ithillel.ua')
fetch_url_profiled('https://lms.ithillel.ua')


Content for "https://google.com":	  b'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="uk"><head><meta content'
Elapsed time (fetch_url): 0.3180389404296875s

Content for "https://ithillel.ua":	  b'\xef\xbb\xbf\n\n\n\n\n\t\n\t\t\n\t\t\t\n\t\t\n\t\t\n\t\n\t\n\t\n\n\n\n\n\n        \n    \t\t\t\t\n\n        \n\t<!DOCTYPE html>\n\t<html lang="uk">\n\t<h'
Elapsed time (fetch_url): 0.24903059005737305s

Content for "https://lms.ithillel.ua":	  b'<!DOCTYPE html><html lang="uk"><head>\n    <meta charset="utf-8">\n    <meta name="viewport" content="'
Elapsed time (fetch_url): 0.18297028541564941s


In [160]:
def profile(f):
    def internal(*args):
        start = time.time()
        f(*args)
        print(f'Elapsed time ({f.__name__}): {time.time() - start}s')
    return internal

fetch_url = profile(fetch_url)
fetch_url('https://ithillel.ua')


Content for "https://ithillel.ua":	  b'\xef\xbb\xbf\n\n\n\n\n\t\n\t\t\n\t\t\t\n\t\t\n\t\t\n\t\n\t\n\t\n\n\n\n\n\n        \n    \t\t\t\t\n\n        \n\t<!DOCTYPE html>\n\t<html lang="uk">\n\t<h'
Elapsed time (fetch_url): 0.21899938583374023s


In [161]:
def fetch_url(url, first_n=None):
    res = requests.get(url)
    print(f'\nContent for "{url}":\t ', res.content[:first_n] if first_n else res.content)

def profile(f):
    def internal(*args, **kwargs):
        start = time.time()
        f(*args, **kwargs)
        print(f'Elapsed time ({f.__name__}): {time.time() - start}s')
    return internal

fetch_url = profile(fetch_url)
fetch_url('https://ithillel.ua', first_n=42)
fetch_url('https://ithillel.ua', first_n=100)
fetch_url('https://ithillel.ua', first_n=1024)


Content for "https://ithillel.ua":	  b'\xef\xbb\xbf\n\n\n\n\n\t\n\t\t\n\t\t\t\n\t\t\n\t\t\n\t\n\t\n\t\n\n\n\n\n\n        '
Elapsed time (fetch_url): 0.22302794456481934s

Content for "https://ithillel.ua":	  b'\xef\xbb\xbf\n\n\n\n\n\t\n\t\t\n\t\t\t\n\t\t\n\t\t\n\t\n\t\n\t\n\n\n\n\n\n        \n    \t\t\t\t\n\n        \n\t<!DOCTYPE html>\n\t<html lang="uk">\n\t<h'
Elapsed time (fetch_url): 0.2219696044921875s

Content for "https://ithillel.ua":	  b'\xef\xbb\xbf\n\n\n\n\n\t\n\t\t\n\t\t\t\n\t\t\n\t\t\n\t\n\t\n\t\n\n\n\n\n\n        \n    \t\t\t\t\n\n        \n\t<!DOCTYPE html>\n\t<html lang="uk">\n\t<head>\n\n\t\t<link rel="preload" href="https://assets.ithillel.ua/fonts/solomon_sans_normal.woff2" as="font" type="font/woff2" crossorigin="anonymous">\n<link rel="preload" href="https://assets.ithillel.ua/fonts/solomon_sans_semibold.woff2" as="font" type="font/woff2" crossorigin="anonymous">\n<link rel="preload" href="https://assets.ithillel.ua/fonts/solomon_sans_bold.woff2" as="font" type="f

In [166]:
def fetch_url(url, first_n=100):
    res = requests.get(url)
    return res.content[:first_n] if first_n else res.content

fetch_url = profile(fetch_url)
with open('content.html', 'w+') as f:
    f.write(str(fetch_url('https://ithillel.ua', first_n=42)))

# cat /tmp/content.html


Elapsed time for function fetch_url with params ('https://ithillel.ua',), {'first_n': 42}: 0.21899914741516113ms


In [167]:
def profile(f):
    def internal(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        print(f'Elapsed time for function {f.__name__} with params {args}, {kwargs}: {time.time() - start}ms')
        return result
    return internal

fetch_url = profile(fetch_url)
with open('content_1.html', 'w+') as f:
    f.write(str(fetch_url('https://ithillel.ua', first_n=42)))


Elapsed time for function fetch_url with params ('https://ithillel.ua',), {'first_n': 42}: 0.21620583534240723ms
Elapsed time for function internal with params ('https://ithillel.ua',), {'first_n': 42}: 0.21620583534240723ms


In [168]:
@profile # -> fetch_url = profile(fetch_url)
def fetch_url(url, first_n=100):
    res = requests.get(url)
    return res.content[:first_n] if first_n else res.content
    
print(fetch_url('https://ithillel.ua', first_n=42))
print(fetch_url('https://google.com'))

Elapsed time for function fetch_url with params ('https://ithillel.ua',), {'first_n': 42}: 0.22803044319152832ms
b'\xef\xbb\xbf\n\n\n\n\n\t\n\t\t\n\t\t\t\n\t\t\n\t\t\n\t\n\t\n\t\n\n\n\n\n\n        '
Elapsed time for function fetch_url with params ('https://google.com',), {}: 0.3059689998626709ms
b'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="uk"><head><meta content'


In [172]:
def test(a, b):
    """ This function is make sum of two int """
    return a+b

help(test)

Help on function test in module __main__:

test(a, b)
    This function is make sum of two int



### We need to go deeper

In [175]:
def profile(f):
    def internal(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        print(f'Elapsed time for function {f.__name__} with params {args}, {kwargs}: {time.time() - start}ms')
        return result
    return internal


@profile
def fetch_url(url, first_n=100):
    """Fetch a given url"""
    res = requests.get(url)
    return res.content[:first_n] if first_n else res.content

help(fetch_url)

Help on function internal in module __main__:

internal(*args, **kwargs)



In [177]:
def profile(f):
    @functools.wraps(f)
    def internal(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        print(f'Elapsed time for function {f.__name__} with params {args}, {kwargs}: {time.time() - start}ms')
        return result
    return internal


@profile
def fetch_url(url, first_n=100):
    """Fetch a given url"""
    res = requests.get(url)
    return res.content[:first_n] if first_n else res.content


help(fetch_url)

Help on function fetch_url in module __main__:

fetch_url(url, first_n=100)
    Fetch a given url



In [179]:
def profile(msg='Elapsed time'):
    def internal(f):
        @functools.wraps(f)
        def deco(*args, **kwargs):
            start = time.time()
            result = f(*args, **kwargs)
            print(msg, f'({f.__name__}): {time.time() - start}s')
            return result
        return deco
    return internal

In [182]:
@profile(msg='час пройшов')
def fetch_url(url, first_n=100):
    """Fetch a given url"""
    res = requests.get(url)
    return res.content[:first_n] if first_n else res.content

fetch_url('https://google.com')
    
@profile(msg='Elapsed time')
def fetch_url(url, first_n=100):
    """Fetch a given url"""
    res = requests.get(url)
    return res.content[:first_n] if first_n else res.content

fetch_url('https://google.com')

час пройшов (fetch_url): 0.3090236186981201s
Elapsed time (fetch_url): 0.3189737796783447s


b'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="uk"><head><meta content'

In [69]:
@profile('Elapsed time') # -> profile_ = profile('Time spent')
                         # -> foo5 = profile_(foo5)
def foo():
    """Help for foo"""
    return 42

help(foo)
print("RESULT: ", foo())

Help on function foo in module __main__:

foo()
    Help for foo

Elapsed time (foo): 0.0s
RESULT:  42


### Template

#### 2-level decorator

In [117]:
def profile(f):
    @functools.wraps(f)
    def deco(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        print('Elapsed time', f'({f.__name__}): {time.time() - start}s')
        return result
    return deco

@profile
def foo(): ...

#### 3-level decorator

In [10]:
def profile(msg='Elapsed time', file=sys.stdout):
    def internal(f):
        @functools.wraps(f)
        def deco(*args, **kwargs):
            start = time.time()
            result = f(*args, **kwargs)
            print(msg, f'({f.__name__}): {time.time() - start}s', file=file)
            return result
        return deco
    return internal

@profile()
def foo(): ...


### Real life examples

In [93]:
# 0 1 2 3 4 5 6 7 8
# 0 1 1 2 3 5 8 13 ...
def fibo(n):
    """Super inefficient fibo function"""
    if n < 2:
        return n
    else:
        return fibo(n-1) + fibo(n-2)
   
fibo(10)

55

In [94]:
@profile()
def fibo(n):
    """Super inefficient fibo function"""
    if n < 2:
        return n
    else:
        return fibo(n-1) + fibo(n-2)
   
fibo(10)

Elapsed time (fibo): 0.0s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 0.0009989738464355469s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 0.0009989738464355469s
Elapsed time (fibo): 0.0009989738464355469s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 0.0009989738464355469s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 0.0s
Elapsed time (fibo): 0.0010013580322265625s
Elapsed time (fibo): 0.0s


55

#### Recursion support

In [95]:
def profile(msg="Elapsed time for function"):
    def internal(f):
        @functools.wraps(f)
        def deco(*args, **kwargs):
            start = time.time()
            deco._num_call += 1
            result = f(*args, **kwargs)
            deco._num_call -= 1
            
            if deco._num_call == 0:
                print(msg, f'{f.__name__}: {time.time() - start}s')
            return result
        
        deco._num_call = 0
        return deco
    
    return internal

@profile()
def fibo(n):
    """Super inefficient fibo function"""
    if n < 2:
        return n
    else:
        return fibo(n-1) + fibo(n-2)
   
fibo(10)

Elapsed time for function fibo: 0.0s


55

#### Exponential backoff

In [185]:
def repeate(max_repeat=10):
    def internal(f):
        @functools.wraps(f)
        def repeater(*args, **kwargs):
            while repeater._num_repeats <= max_repeat:
                try:
                    return f(*args, **kwargs)
                except Exception as ex:
                    if repeater._num_repeats == max_repeat:
                        raise
                    else:
                        print(f'Failed after {repeater._num_repeats+1} times, trying again after {2**repeater._num_repeats} sec...')
                        sleep(2**repeater._num_repeats)
                        repeater._num_repeats += 1
                
        repeater._num_repeats = 0
        return repeater
    return internal


@repeate(max_repeat=4)
# @repeate()
# @repeate # note the difference
def connect_to_server(*args):
    print('Trying to connect: ', *args)
    if sum(random.choices([0, 1], [0.8, 0.2])) == 0:
        raise RuntimeError('Failed to connect')
    print('SUCCESS!')

connect_to_server('google.com')

Trying to connect:  google.com
Failed after 1 times, trying again after 1 sec...
Trying to connect:  google.com
Failed after 2 times, trying again after 2 sec...
Trying to connect:  google.com
SUCCESS!


#### Cache

In [4]:
def cache(f):
    
    @functools.wraps(f)
    def deco(*args):
        
        if args in deco._cache:
            return deco._cache[args]
        
        result = f(*args)
        
        deco._cache[args] = result
        
        return result
    
    deco._cache = {}
        
    return deco


In [186]:
# LRU

def cache(max_limit=64):
    def internal(f):
        @functools.wraps(f)
        def deco(*args, **kwargs):
            cache_key = (args, tuple(kwargs.items()))
            if cache_key in deco._cache:
                # переносимо в кінець списку
                deco._cache.move_to_end(cache_key, last=True)
                return deco._cache[cache_key]
            
            result = f(*args, **kwargs)
            # видаляємо якшо досягли ліміта
            if len(deco._cache) >= max_limit:
                 # видаляємо перший елемент
                deco._cache.popitem(last=False)
            deco._cache[cache_key] = result
            return result
        deco._cache = OrderedDict()
        return deco
    return internal

In [187]:
# @profile(msg='Elapsed time')
# @cache(max_limit=5)
@profile(msg='Elapsed time')
@cache
def fetch_url(url, first_n=100):
    """Fetch a given url"""
    res = requests.get(url)
    return res.content[:first_n] if first_n else res.content

fetch_url('https://google.com')
fetch_url('https://google.com')
fetch_url('https://google.com')
fetch_url('https://ithillel.ua')
fetch_url('https://dou.ua')
fetch_url('https://ain.ua')
fetch_url('https://youtube.com')
# fetch_url(url='https://reddit.com')
# fetch_url._cache

Elapsed time (internal): 0.0s
Elapsed time (internal): 0.0s
Elapsed time (internal): 0.0s
Elapsed time (internal): 0.0s
Elapsed time (internal): 0.0s
Elapsed time (internal): 0.0s
Elapsed time (internal): 0.0s


<function __main__.cache.<locals>.internal.<locals>.deco>

<img src=https://i.imgur.com/IgBFGaR.png>

In [189]:
@profile()
@cache
def fibo(n):
    """Super inefficient fibo function"""
    if n < 2:
        return n
    else:
        return fibo(n-1) + fibo(n-2)

In [190]:
# for i in range(10000):
#     print(i, '->', fibo(i))
print(fibo(500))

Elapsed time for function fibo: 0.0016300678253173828s
139423224561697880139724382870407283950070256587697307264108962948325571622863290691557658876222521294125


In [114]:
# fibo._cache
# id(fibo._cache)

In [128]:
from functools import lru_cache

@lru_cache(maxsize=64)
def foo():
    print('foo')

### Links

1. Python Introduction: https://www.youtube.com/watch?v=5V7XG1mGiHc&list=PLlb7e2G7aSpTTNp7HBYzCBByaE1h54ruW
2. Теорія: http://bit.ly/2z5yatp: стр. 414
3. Exponential backoff: https://bit.ly/3nvOyeQ
4. Установлення jupyter: https://www.youtube.com/watch?v=VIaur9G-0tc